In [1]:
# Make sure we have a GPU at all
!nvidia-smi --gpu-reset

# Remove warnings created constantly...
import sys

import multiprocessing

print(multiprocessing.cpu_count())

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
from numba import cuda 
device = cuda.close()


from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
import tensorflow

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tensorflow.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.Session(config=config))
    
!nvidia-smi

GPU reset unavailable: Insufficient Permissions
96


Using TensorFlow backend.
2022-05-20 09:19:59.057101: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


Fri May 20 09:20:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:39:00.0 Off |                    0 |
| N/A   40C    P0    51W / 350W |      5MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM3...  On   | 00000000:3B:00.0 Off |                    0 |
| N/A   

In [2]:
# Prelimitary defense - install libraries 
!python3 --version

!python3.8 -m pip install neptune-tensorflow-keras
!python3.8 -m pip install neptune-client
#python3.8 -m pip install keras==2.2.4
#!python3 -m pip install -U scikit-image==0.16.2
#!python3 -m pip install h5py==2.10.0
#!python3 -m pip install opencv-python
#!python3 -m pip install scikit-image

Python 3.8.10
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


In [3]:
import os

import sys
import random
import math
import re
import time
import numpy as np
import skimage.draw
import cv2
import json
import matplotlib
import matplotlib.pyplot as plt

# Import Tensorflow after numerous attempts (version control)
import tensorflow as tf
from tensorflow import keras

import neptune.new as neptune
from neptune.new.integrations.tensorflow_keras import NeptuneCallback

run = neptune.init(
    project="hthoft/parcel-training",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJjZWI2Njc1Yy1mYmM1LTRmZDktYTA0OS02NDBjNTI2NjJlNDAifQ==",
)  # your credentials

params = {"lr": 0.002, "momentum": 0.4, "epochs": 10, "batch_size": 64}
run["parameters"] = params

# Check we have our GPU available...
physical_devices = (tf.config.experimental.list_physical_devices('GPU'))
print("Found:  " + str(physical_devices))

# Root directory of the project
ROOT_DIR = os.path.abspath("/user/student.aau.dk/htra19/group660/tests")
print("Directory: " +ROOT_DIR)

# Import Mask RCNN
sys.path.append("/user/student.aau.dk/htra19/group660/tests/Mask_RCNN")  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)
    
# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

https://app.neptune.ai/hthoft/parcel-training/e/PAR-21
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Found:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]
Directory: /user/student.aau.dk/htra19/group660/tests


2022-05-20 09:20:16.108182: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-20 09:20:16.112242: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 0 with properties: 
name: Tesla V100-SXM3-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.597
pciBusID: 0000:39:00.0
2022-05-20 09:20:16.119068: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 1 with properties: 
name: Tesla V100-SXM3-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.597
pciBusID: 0000:3b:00.0
2022-05-20 09:20:16.123006: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 2 with properties: 
name: Tesla V100-SXM3-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.597
pciBusID: 0000:5e:00.0
2022-05-20 09:20:16.126932: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 3 with properties: 
name: Tesla V100-SXM3-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.597
pciBusID: 0000:b7:00.0
202

In [4]:
class CustomConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "object"
    
    BACKBONE = "resnet101"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 4
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 2  # background + 1 parcel

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    
    # Input image resizing:
    IMAGE_RESIZE_MODE = "square"
    IMAGE_MIN_DIM = 1024
    IMAGE_MAX_DIM = 1024
    
    LEARNING_RATE = 0.002

    # The strides of each layer of the FPN Pyramid. These values
    # are based on a Resnet101 backbone.
    BACKBONE_STRIDES = [4, 8, 16, 32, 64]
    
    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 1024
    
    POST_NMS_ROIS_INFERENCE = 2000
    
    MAX_GT_INSTANCES = 20

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 1000

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 29
    
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.7
    
config = CustomConfig()
config.display()

#run['Model Config Pars'] = str(config.to_dict())


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     8
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      4
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  1024
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.002
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE        

In [5]:
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Dog-Cat dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("object", 1, "box")

        self.add_class("object", 2, "bag")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # VGG Image Annotator saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        annotations1 = json.load(open(os.path.join(dataset_dir, "annotations.json")))
        # print(annotations1)
        annotations = list(annotations1.values())  # don't need the dict keys

         # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]

        
        print(annotations)
        # Add images
        for a in annotations:

            print(a)
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['name'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"box": 1,"bag": 2}
            print(name_dict)
            # key = tuple(name_dict)
            num_ids = [name_dict[a] for a in objects]
     
            # num_ids = [int(n['Event']) for n in objects]
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Dog-Cat dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

neptune_cbk = NeptuneCallback(run=run, base_namespace="metrics")
            
class NeptuneMonitor(keras.callbacks.Callback):
     def on_epoch_end(self, epoch, logs=None):
          for metric_name, metric_value in logs.items():
              run["train/{}".format(metric_name)].log(metric_value)



In [6]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    #dataset_train.load_custom("/user/student.aau.dk/htra19/group660/tests/dataset_first_test", "train")
    dataset_train.load_custom("/user/student.aau.dk/htra19/group660/tests/dataset_beumer_test", "train")
    #dataset_train.load_custom("/user/student.aau.dk/htra19/group660/tests/dataset_final_run", "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    #dataset_val.load_custom("/user/student.aau.dk/htra19/group660/tests/dataset_first_test", "val")
    dataset_val.load_custom("/user/student.aau.dk/htra19/group660/tests/dataset_beumer_test", "val")
    #dataset_val.load_custom("/user/student.aau.dk/htra19/group660/tests/dataset_final_run", "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=10,
                layers='all',
                custom_callbacks=[neptune_cbk])

In [7]:
#strategy = tf.distribute.MirroredStrategy()

config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config, model_dir=DEFAULT_LOGS_DIR)




In [8]:
# Which weights to start with?
init_with = "last"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    strategy = tf.distribute.MirroredStrategy()
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)
    


2022-05-20 09:20:29.898267: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2700000000 Hz
2022-05-20 09:20:29.899132: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1f1cd630 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-05-20 09:20:29.899157: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-05-20 09:20:32.756315: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2291cd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-05-20 09:20:32.756361: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM3-32GB, Compute Capability 7.0
2022-05-20 09:20:32.756368: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Tesla V100-SXM3-32GB, Compute Capability 7.0
2022-05-20 09:20:32.756373: I tensorflow/compiler/xla/service/service.cc:17

2022-05-20 09:20:37.902214: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1206] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-05-20 09:20:37.902270: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1212]      0 1 2 3 
2022-05-20 09:20:37.902278: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1225] 0:   N Y Y Y 
2022-05-20 09:20:37.902283: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1225] 1:   Y N Y Y 
2022-05-20 09:20:37.902288: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1225] 2:   Y Y N Y 
2022-05-20 09:20:37.902292: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1225] 3:   Y Y Y N 
2022-05-20 09:20:37.932728: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1351] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 31004 MB memory) -> physical GPU (device: 0, name: Tesla V100-SXM3-32GB, pci bus id: 0000:39:00.0, compute capability: 7.0)
2022-05-20 09:20:37.940060: I tensorflow/core/common_runtime/gpu/gpu_d


Re-starting from epoch 10


In [9]:
train(model)

[{'filename': '000.jpeg', 'size': 507954, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [878, 870, 766, 754, 757, 774, 811, 827, 836, 839, 846, 883, 939, 948, 957, 962, 962, 960, 950, 931, 879], 'all_points_y': [445, 452, 519, 528, 532, 559, 612, 637, 647, 651, 649, 626, 587, 581, 571, 567, 561, 553, 545, 515, 446]}, 'region_attributes': {'name': 'box'}}], 'file_attributes': {}}, {'filename': '001.jpeg', 'size': 508157, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [876, 865, 809, 753, 763, 804, 831, 841, 861, 889, 939, 955, 964, 959, 953, 885, 879], 'all_points_y': [444, 455, 491, 528, 543, 601, 642, 651, 642, 622, 588, 574, 565, 551, 550, 451, 446]}, 'region_attributes': {'name': 'box'}}], 'file_attributes': {}}, {'filename': '004.jpeg', 'size': 507288, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [710, 709, 712, 718, 718, 652, 647, 640, 639, 640], 'all_points_y': [453, 466, 520, 604, 618, 622, 597, 541, 492, 456]}

{'filename': '007.jpeg', 'size': 511007, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [963, 987, 1077, 1054], 'all_points_y': [225, 329, 309, 204]}, 'region_attributes': {'name': 'box'}}], 'file_attributes': {}}
objects: ['box']
{'box': 1, 'bag': 2}
numids [1]
{'filename': '008.jpeg', 'size': 503706, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [837, 828, 923, 927], 'all_points_y': [332, 436, 442, 336]}, 'region_attributes': {'name': 'box'}}], 'file_attributes': {}}
objects: ['box']
{'box': 1, 'bag': 2}
numids [1]
{'filename': '009.jpeg', 'size': 512297, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [761, 717, 814, 854], 'all_points_y': [602, 687, 733, 650]}, 'region_attributes': {'name': 'box'}}], 'file_attributes': {}}
objects: ['box']
{'box': 1, 'bag': 2}
numids [1]
{'filename': '010.jpeg', 'size': 520970, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [1070, 1055, 780, 800], 'all_points_y':

{'filename': '033.jpeg', 'size': 510775, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [833, 797, 762, 726, 692, 737, 757, 792, 832, 873, 947, 1015, 1014, 1000], 'all_points_y': [212, 246, 280, 314, 345, 392, 416, 452, 494, 539, 469, 407, 401, 389]}, 'region_attributes': {'name': 'box'}}], 'file_attributes': {}}
objects: ['box']
{'box': 1, 'bag': 2}
numids [1]
{'filename': '034.jpeg', 'size': 503924, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [820, 747, 929, 1003], 'all_points_y': [304, 554, 607, 358]}, 'region_attributes': {'name': 'box'}}], 'file_attributes': {}}
objects: ['box']
{'box': 1, 'bag': 2}
numids [1]
{'filename': '035.jpeg', 'size': 505976, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [1025, 953, 885, 838, 878, 918, 957, 971, 1014, 1081, 1130, 1158], 'all_points_y': [322, 393, 460, 504, 545, 585, 626, 641, 598, 534, 486, 459]}, 'region_attributes': {'name': 'box'}}], 'file_attributes': {}}
objects: ['

{'filename': '059.jpeg', 'size': 503959, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [984, 1043, 966, 907, 911], 'all_points_y': [595, 660, 732, 663, 658]}, 'region_attributes': {'name': 'box'}}], 'file_attributes': {}}
objects: ['box']
{'box': 1, 'bag': 2}
numids [1]
{'filename': '060.jpeg', 'size': 527931, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [911, 1022, 793, 683, 687, 794], 'all_points_y': [260, 431, 578, 407, 403, 337]}, 'region_attributes': {'name': 'box'}}], 'file_attributes': {}}
objects: ['box']
{'box': 1, 'bag': 2}
numids [1]
{'filename': '061.jpeg', 'size': 521132, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [792, 996, 986, 833, 782], 'all_points_y': [125, 134, 405, 402, 398]}, 'region_attributes': {'name': 'box'}}], 'file_attributes': {}}
objects: ['box']
{'box': 1, 'bag': 2}
numids [1]
{'filename': '062.jpeg', 'size': 520900, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x':

{'filename': '084.jpeg', 'size': 505413, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [764, 1017, 981, 725], 'all_points_y': [572, 623, 812, 760]}, 'region_attributes': {'name': 'box'}}], 'file_attributes': {}}
objects: ['box']
{'box': 1, 'bag': 2}
numids [1]
{'filename': '085.jpeg', 'size': 500850, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [622, 880, 907, 649], 'all_points_y': [220, 187, 375, 410]}, 'region_attributes': {'name': 'box'}}], 'file_attributes': {}}
objects: ['box']
{'box': 1, 'bag': 2}
numids [1]
{'filename': '086.jpeg', 'size': 506212, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [836, 873, 875, 883, 935, 992, 1010, 1016, 1017, 1008, 984, 975, 974, 926, 868, 836, 834], 'all_points_y': [390, 263, 256, 256, 272, 288, 294, 299, 311, 341, 418, 439, 443, 429, 410, 399, 397]}, 'region_attributes': {'name': 'box'}}], 'file_attributes': {}}
objects: ['box']
{'box': 1, 'bag': 2}
numids [1]
{'filename': '08

...

{'filename': '852.jpeg', 'size': 541767, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [774, 782, 962, 1018, 1040, 1068, 1093, 997, 900, 847, 832, 800, 776], 'all_points_y': [320, 316, 247, 225, 284, 354, 417, 456, 493, 512, 474, 393, 328]}, 'region_attributes': {'name': 'box'}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [930, 848, 724, 806], 'all_points_y': [750, 834, 713, 629]}, 'region_attributes': {'name': 'box'}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [815, 825, 947, 950, 949, 937, 829, 817], 'all_points_y': [525, 521, 518, 522, 642, 646, 645, 634]}, 'region_attributes': {'name': 'box'}}], 'file_attributes': {}}
objects: ['box', 'box', 'box']
{'box': 1, 'bag': 2}
numids [1, 1, 1]
{'filename': '862.jpeg', 'size': 572954, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [997, 1060, 1065, 1067, 1072, 1078, 1079, 1068, 1023, 915, 841, 830, 815, 812, 809, 804, 801, 802, 806, 815, 829], 'all_points_y': [274, 27

{'filename': '949.jpeg', 'size': 534833, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [858, 939, 1001, 1018, 1025, 1036, 1043, 1043, 1039, 1022, 960, 899, 872, 867, 865, 859, 848, 840, 840], 'all_points_y': [540, 524, 511, 507, 535, 595, 627, 636, 637, 639, 653, 664, 669, 672, 670, 641, 583, 550, 542]}, 'region_attributes': {'name': 'box'}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [747, 693, 687, 675, 673, 673, 669, 653, 630, 624, 619, 619, 622, 625, 627, 631, 692, 732, 736, 745, 781, 793, 790, 784], 'all_points_y': [503, 476, 472, 470, 473, 476, 480, 510, 551, 562, 569, 570, 575, 579, 581, 584, 616, 637, 634, 622, 556, 531, 530, 524]}, 'region_attributes': {'name': 'box'}}], 'file_attributes': {}}
objects: ['box', 'box']
{'box': 1, 'bag': 2}
numids [1, 1]
{'filename': '501.jpeg', 'size': 515310, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [666, 679, 700, 704, 764, 777, 782, 802, 819, 833, 850, 870, 892, 916, 927, 931, 9

{'filename': '645.jpeg', 'size': 547255, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [892, 868, 867, 857, 850, 834, 824, 813, 811, 797, 791, 757, 751, 736, 722, 712, 695, 689, 685, 672, 671, 675, 678, 696, 703, 705, 719, 764, 779, 791, 802, 820, 829, 832, 841, 856, 891, 904, 912, 932, 937, 947, 952, 983, 1029, 1040, 1062, 1068, 1077, 1083, 1094, 1098, 1079, 1066, 1057, 1046, 1042, 1027, 1015, 997, 980, 969, 969, 951, 938, 930, 930, 919, 917, 910, 905, 896], 'all_points_y': [272, 289, 293, 297, 303, 309, 312, 321, 324, 331, 328, 342, 350, 352, 357, 348, 344, 345, 353, 355, 361, 364, 374, 408, 420, 434, 451, 510, 538, 559, 576, 599, 602, 618, 643, 680, 655, 640, 639, 626, 623, 613, 610, 588, 560, 551, 539, 535, 530, 527, 525, 523, 502, 492, 485, 478, 463, 449, 439, 421, 403, 391, 385, 365, 348, 336, 332, 321, 311, 300, 290, 276]}, 'region_attributes': {'name': 'bag'}}], 'file_attributes': {}}
objects: ['bag']
{'box': 1, 'bag': 2}
numids [2]
{'filename': '652.jpeg

{'filename': '725.jpeg', 'size': 545794, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [632, 654, 668, 670, 673, 679, 681, 700, 705, 717, 733, 747, 782, 793, 838, 846, 849, 850, 846, 833, 826, 817, 806, 796, 787, 777, 773, 781, 778, 776, 774, 765, 755, 739, 723, 711, 691, 673, 654, 640, 623, 611, 608, 607, 609, 616], 'all_points_y': [324, 398, 446, 458, 465, 476, 482, 476, 471, 473, 464, 451, 436, 435, 424, 421, 409, 404, 397, 376, 356, 336, 311, 274, 245, 221, 210, 198, 191, 190, 196, 199, 203, 211, 218, 227, 236, 243, 244, 247, 250, 254, 256, 264, 268, 275]}, 'region_attributes': {'name': 'bag'}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [1165, 1194, 1183, 1158, 1142, 1087, 1043, 1031, 1020, 1018, 996, 986, 980, 970, 954, 938, 915, 895, 884, 875, 868, 849, 839, 824, 847, 860, 866, 869, 892, 920, 979, 995, 1011, 1026, 1127], 'all_points_y': [608, 556, 548, 537, 530, 494, 470, 461, 457, 455, 448, 453, 462, 481, 512, 535, 561, 590, 605, 624, 627, 6

{'filename': '961.jpeg', 'size': 511966, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [920, 963, 977, 992, 1001, 1009, 1034, 1047, 1061, 1078, 1047, 1022, 1000, 948, 908, 841, 806, 768, 745, 722, 708, 695, 684, 681, 721, 745, 799], 'all_points_y': [375, 422, 439, 454, 462, 471, 495, 508, 522, 537, 576, 606, 629, 678, 716, 787, 752, 710, 682, 661, 644, 623, 618, 614, 573, 550, 492]}, 'region_attributes': {'name': 'bag'}}], 'file_attributes': {}}
objects: ['bag']
{'box': 1, 'bag': 2}
numids [2]
{'filename': '967.jpeg', 'size': 504982, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [897, 921, 932, 951, 1007, 1007, 1021, 1011, 991, 932, 908, 896, 885, 870, 858, 847, 836, 831, 806, 803, 796, 773, 766, 745, 723, 698, 685, 675, 679, 706, 713, 729, 756, 800, 850, 874, 887, 889, 899, 896], 'all_points_y': [305, 325, 344, 391, 452, 465, 462, 502, 519, 557, 578, 586, 596, 607, 618, 625, 631, 649, 652, 664, 669, 625, 620, 594, 563, 527, 491, 497, 470, 

In [10]:
!nvidia-smi

Fri May 20 09:22:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:39:00.0 Off |                    0 |
| N/A   42C    P0    69W / 350W |  31452MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM3...  On   | 00000000:3B:00.0 Off |                    0 |
| N/A   